### Sample script to read the text in wiki and answer the questions.

In [3]:
# Library imports
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer, TFBertForQuestionAnswering
import numpy as np

2024-05-31 08:22:46.561094: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 08:22:46.561536: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 08:22:46.563790: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 08:22:46.592894: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 08:22:47.135005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [4]:
# Load pre-trained BERT model and tokenizer from Hugging Face
model = TFBertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [5]:
# Core function for interaction
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='tf', max_length=512, truncation=True)
    outputs = model(inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
    answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

In [6]:
# Example usage
context = "Wikipedia is a free online encyclopedia, created and edited by volunteers around the world and hosted by the Wikimedia Foundation."
question = "What is Wikipedia?"

In [7]:
# Results
answer = answer_question(question, context)
print(f"Answer: {answer}")

Answer: a free online encyclopedia


In [8]:
question = "Who found wikipedia?"
answer = answer_question(question, context)
print(answer)

volunteers


In [9]:
# tinkering a bit
context = """
‘The Dumb Man’
by Sherwood Anderson
There is a story - I cannot tell it - I have no words. The story is almost
forgotten but sometimes I remember.
The story concerns three men in a house in a street. If I could say the
words I would sing the story. I would whisper it into the ears of women,
of mothers. I would run through the streets saying it over and over. My
tongue would be torn loose--it would rattle against my teeth.
The three men are in a room in the house. One is young and dandified.
He continually laughs.
There is a second man who has a long white beard. He is consumed with
doubt but occasionally his doubt leaves him and he sleeps.
A third man there is who has wicked eyes and who moves nervously
about the room rubbing his hands together. The three men are waiting -
waiting.
Upstairs in the house there is a woman standing with her back to a wall,
in half darkness by a window.
That is the foundation of my story and everything I will ever know is
distilled in it.
I remember that a fourth man came to the house, a white silent man.
Everything was as silent as the sea at night. His feet on the stone floor of
the room where the three men were made no sound.
The man with the wicked eyes became like a boiling liquid - he ran back
and forth like a caged animal. The old grey man was infected by his
nervousness - he kept pulling at his beard.
The fourth man, the white one, went upstairs to the woman.
There she was - waiting.
How silent the house was - how loudly all the clocks in the neighbourhood
ticked. The woman upstairs craved love. That must have been the story.
She hungered for love with her whole being. She wanted to create in love.
When the white silent man came into her presence she sprang forward.
Her lips were parted. There was a smile on her lips.
The white one said nothing. In his eyes there was no rebuke, no question.
His eyes were as impersonal as stars.
Down stairs the wicked one whined and ran back and forth like a little lost
hungry dog. The grey one tried to follow him about but presently grew
tired and lay down on the floor to sleep. He never awoke again.
The dandified fellow lay on the floor too. He laughed and played with his
tiny black moustache.
I have no words to tell what happened in my story. I cannot tell the story.
The white silent one may have been Death.
The waiting eager woman may have been Life.
Both the old grey bearded man and the wicked one puzzle me. I think and
think but cannot understand them. Most of the time however I do not
think of them at all. I keep thinking about the dandified man who laughed
all through my story.
If I could understand him I could understand everything. I could run
through the world telling a wonderful story. I would no longer be dumb.
Why was I not given words? Why am I dumb?
I have a wonderful story to tell, but know no way to tell it.
"""

In [10]:
question = "Who is dumbman?"
answer = answer_question(question, context)
print(answer)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


the dumb man
